In [1]:
import os
import json
import pickle
import random
from collections import defaultdict, Counter

from indra.literature.adeft_tools import universal_extract_text
from indra.databases.hgnc_client import get_hgnc_name, get_hgnc_id

from adeft.discover import AdeftMiner
from adeft.gui import ground_with_gui
from adeft.modeling.label import AdeftLabeler
from adeft.modeling.classify import AdeftClassifier
from adeft.disambiguate import AdeftDisambiguator, load_disambiguator


from adeft_indra.ground.ground import AdeftGrounder
from adeft_indra.model_building.s3 import model_to_s3
from adeft_indra.model_building.escape import escape_filename
from adeft_indra.db.content import get_pmids_for_agent_text, get_pmids_for_entity, \
    get_plaintexts_for_pmids

In [2]:
adeft_grounder = AdeftGrounder()

In [3]:
shortforms = ['LH']
model_name = ':'.join(sorted(escape_filename(shortform) for shortform in shortforms))
results_path = os.path.abspath(os.path.join('../..', 'results', model_name))

In [4]:
miners = dict()
all_texts = {}
for shortform in shortforms:
    pmids = get_pmids_for_agent_text(shortform)
    text_dict = get_plaintexts_for_pmids(pmids, contains=shortforms)
    text_dict = {pmid: text for pmid, text in text_dict.items() if len(text) > 5}
    miners[shortform] = AdeftMiner(shortform)
    miners[shortform].process_texts(text_dict.values())
    all_texts.update(text_dict)

longform_dict = {}
for shortform in shortforms:
    longforms = miners[shortform].get_longforms()
    longforms = [(longform, count, score) for longform, count, score in longforms
                 if count*score > 2]
    longform_dict[shortform] = longforms
    
combined_longforms = Counter()
for longform_rows in longform_dict.values():
    combined_longforms.update({longform: count for longform, count, score
                               in longform_rows})
grounding_map = {}
names = {}
for longform in combined_longforms:
    groundings = adeft_grounder.ground(longform)
    if groundings:
        grounding = groundings[0]['grounding']
        grounding_map[longform] = grounding
        names[grounding] = groundings[0]['name']
longforms, counts = zip(*combined_longforms.most_common())
pos_labels = []

In [5]:
list(zip(longforms, counts))

[('luteinizing hormone', 4666),
 ('lateral hypothalamus', 308),
 ('luteinising hormone', 202),
 ('learned helplessness', 45),
 ('lateral hypothalamic', 40),
 ('left hemisphere', 34),
 ('lutenizing hormone', 27),
 ('leutinizing hormone', 19),
 ('lateral horn', 15),
 ('lysyl hydroxylase', 15),
 ('luteinizing hor mone', 13),
 ('lateral hypothalamic area', 12),
 ('light harvesting', 11),
 ('lipohypertrophy', 9),
 ('lymphoid follicle hyperplasia', 9),
 ('leonurine hydrochloride', 8),
 ('left hand', 7),
 ('lipid hydroperoxide', 7),
 ('lymphocytic hypophysitis', 7),
 ('linker histone', 6),
 ('linear headache', 6),
 ('lyon hypertensive', 6),
 ('lianhuaqingwen', 6),
 ('leuteinizing hormone', 6),
 ('liver hydrolysate', 5),
 ('laparoscopic hepatectomy', 5),
 ('laparoscopic hysterectomy', 5),
 ('lower hypocotyl', 5),
 ('hormone lutropin', 5),
 ('left cerebral hemisphere', 5),
 ('lutenising hormone', 5),
 ('l', 5),
 ('liposomal honokiol', 4),
 ('local heat', 4),
 ('lidocaine hydrochloride', 4),
 ('

In [6]:
try:
    disamb = load_disambiguator(shortforms[0])
    for shortform, gm in disamb.grounding_dict.items():
        for longform, grounding in gm.items():
            grounding_map[longform] = grounding
    for grounding, name in disamb.names.items():
        names[grounding] = name
    pos_labels = disamb.pos_labels
except Exception:
    pass

In [7]:
names

{'FPLX:LH': 'LH',
 'MESH:D007026': 'Hypothalamic Area, Lateral',
 'CHEBI:CHEBI:24621': 'hormone',
 'MESH:D006380': 'Helplessness, Learned',
 'MESH:D006965': 'Hyperplasia',
 'CHEBI:CHEBI:36807': 'hydrochloride',
 'CHEBI:CHEBI:61051': 'lipid hydroperoxide',
 'MESH:D006261': 'Headache',
 'MESH:C000588632': 'lianhuaqingwen',
 'MESH:D006498': 'Hepatectomy',
 'MESH:D007044': 'Hysterectomy',
 'MESH:D018546': 'Hypocotyl',
 'CHEBI:CHEBI:81568': 'Luteinizing hormone',
 'CHEBI:CHEBI:15603': 'L-leucine',
 'MESH:C533735': 'liposomal honokiol',
 'CHEBI:CHEBI:50512': 'lidocaine hydrochloride',
 'MESH:D019262': 'Habenula',
 'MESH:D006664': 'History',
 'CHEBI:CHEBI:18421': 'superoxide',
 'MESH:D008138': 'Loop of Henle',
 'MESH:D006495': 'Heparin, Low-Molecular-Weight'}

In [8]:
grounding_map, names, pos_labels = ground_with_gui(longforms, counts, 
                                                   grounding_map=grounding_map,
                                                   names=names, pos_labels=pos_labels, no_browser=True, port=8891)

INFO: [2020-11-10 02:33:21] indra.ontology.bio.ontology - Loading INDRA bio ontology from cache at /home/ubuntu/.indra/bio_ontology/1.4/bio_ontology.pkl


In [9]:
result = [grounding_map, names, pos_labels]

In [10]:
result

[{'hormone lutropin': 'CHEBI:CHEBI:81568',
  'l': 'ungrounded',
  'laparoscopic hepatectomy': 'ungrounded',
  'laparoscopic hysterectomy': 'ungrounded',
  'lateral habenula': 'ungrounded',
  'lateral horn': 'ungrounded',
  'lateral hypothalamic': 'MESH:D007026',
  'lateral hypothalamic area': 'MESH:D007026',
  'lateral hypothalamic nucleus': 'MESH:D007026',
  'lateral hypothalamus': 'MESH:D007026',
  'learned helplessness': 'ungrounded',
  'left cerebral hemisphere': 'ungrounded',
  'left hand': 'ungrounded',
  'left hemisphere': 'ungrounded',
  'length of hypocotyl': 'ungrounded',
  'leonurine hydrochloride': 'PUBCHEM:161464',
  'leuteinizing hormone': 'FPLX:LH',
  'leutenizing hormone': 'CHEBI:CHEBI:81568',
  'leutinizing hormone': 'FPLX:LH',
  'lh': 'ungrounded',
  'lianhuaqingwen': 'MESH:C000588632',
  'lidocaine hydrochloride': 'CHEBI:CHEBI:50512',
  'life history': 'ungrounded',
  'light harvesting': 'ungrounded',
  'linear headache': 'ungrounded',
  'linker histone': 'ungrounded

In [11]:
grounding_map, names, pos_labels = [{'hormone lutropin': 'FPLX:LH',
  'l': 'ungrounded',
  'laparoscopic hepatectomy': 'ungrounded',
  'laparoscopic hysterectomy': 'ungrounded',
  'lateral habenula': 'ungrounded',
  'lateral horn': 'ungrounded',
  'lateral hypothalamic': 'MESH:D007026',
  'lateral hypothalamic area': 'MESH:D007026',
  'lateral hypothalamic nucleus': 'MESH:D007026',
  'lateral hypothalamus': 'MESH:D007026',
  'learned helplessness': 'ungrounded',
  'left cerebral hemisphere': 'ungrounded',
  'left hand': 'ungrounded',
  'left hemisphere': 'ungrounded',
  'length of hypocotyl': 'ungrounded',
  'leonurine hydrochloride': 'PUBCHEM:161464',
  'leuteinizing hormone': 'FPLX:LH',
  'leutenizing hormone': 'FPLX:LH',
  'leutinizing hormone': 'FPLX:LH',
  'lh': 'ungrounded',
  'lianhuaqingwen': 'MESH:C000588632',
  'lidocaine hydrochloride': 'CHEBI:CHEBI:50512',
  'life history': 'ungrounded',
  'light harvesting': 'ungrounded',
  'linear headache': 'ungrounded',
  'linker histone': 'ungrounded',
  'lipid hydroperoxide': 'CHEBI:CHEBI:61051',
  'lipid hyperoxide': 'ungrounded',
  'lipohypertrophy': 'ungrounded',
  'liposomal honokiol': 'MESH:C533735',
  'liver hydrolysate': 'ungrounded',
  'local heat': 'ungrounded',
  'loop of henle': 'MESH:D008138',
  'low and one high': 'ungrounded',
  'low molecular weight heparin': 'MESH:D006495',
  'lower hypocotyl': 'MESH:D018546',
  'luteinising hormone': 'FPLX:LH',
  'luteinizing hor mone': 'FPLX:LH',
  'luteinizing hormone': 'FPLX:LH',
  'luteinsing hormone': 'FPLX:LH',
  'luteinzing hormone': 'FPLX:LH',
  'lutenising hormone': 'FPLX:LH',
  'lutenizing hormone': 'FPLX:LH',
  'lymphocytic hypophysitis': 'ungrounded',
  'lymphoid follicle hyperplasia': 'ungrounded',
  'lyon hypertensive': 'ungrounded',
  'lysyl hydroxylase': 'PLOD'},
 {'MESH:D007026': 'Hypothalamic Area, Lateral',
  'PUBCHEM:161464': 'Leonurine',
  'FPLX:LH': 'LH',
  'MESH:C000588632': 'lianhuaqingwen',
  'CHEBI:CHEBI:50512': 'lidocaine hydrochloride',
  'CHEBI:CHEBI:61051': 'lipid hydroperoxide',
  'MESH:C533735': 'liposomal honokiol',
  'MESH:D008138': 'Loop of Henle',
  'MESH:D006495': 'Heparin, Low-Molecular-Weight',
  'MESH:D018546': 'Hypocotyl',
  'PLOD': 'PLOD'},
 ['FPLX:LH']]

In [12]:
excluded_longforms = ['lh', 'l']

In [13]:
grounding_dict = {shortform: {longform: grounding_map[longform] 
                              for longform, _, _ in longforms if longform in grounding_map
                              and longform not in excluded_longforms}
                  for shortform, longforms in longform_dict.items()}
result = [grounding_dict, names, pos_labels]

if not os.path.exists(results_path):
    os.mkdir(results_path)
with open(os.path.join(results_path, f'{model_name}_preliminary_grounding_info.json'), 'w') as f:
    json.dump(result, f)

In [14]:
additional_entities = {}

In [15]:
unambiguous_agent_texts = {}

In [16]:
labeler = AdeftLabeler(grounding_dict)
corpus = labeler.build_from_texts((text, pmid) for pmid, text in all_texts.items())
agent_text_pmid_map = defaultdict(list)
for text, label, id_ in corpus:
    agent_text_pmid_map[label].append(id_)

entity_pmid_map = {entity: set(get_pmids_for_entity(*entity.split(':', maxsplit=1),
                                                major_topic=True))for entity in additional_entities}

In [18]:
intersection1 = []
for entity1, pmids1 in entity_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection1.append((entity1, entity2, len(pmids1 & pmids2)))

In [19]:
intersection2 = []
for entity1, pmids1 in agent_text_pmid_map.items():
    for entity2, pmids2 in entity_pmid_map.items():
        intersection2.append((entity1, entity2, len(set(pmids1) & pmids2)))

In [20]:
intersection1

[('HGNC:10967', 'HGNC:10967', 98)]

In [21]:
intersection2

[('GO:GO:0001837', 'HGNC:10967', 0),
 ('ungrounded', 'HGNC:10967', 0),
 ('HGNC:10967', 'HGNC:10967', 0),
 ('MESH:D055032', 'HGNC:10967', 0)]

In [22]:
all_used_pmids = set()
for entity, agent_texts in unambiguous_agent_texts.items():
    used_pmids = set()
    for agent_text in agent_texts[1]:
        pmids = set(get_pmids_for_agent_text(agent_text))
        new_pmids = list(pmids - all_texts.keys() - used_pmids)
        text_dict = get_plaintexts_for_pmids(new_pmids, contains=agent_texts)
        corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])
        used_pmids.update(new_pmids)
    all_used_pmids.update(used_pmids)
        
for entity, pmids in entity_pmid_map.items():
    new_pmids = list(set(pmids) - all_texts.keys() - all_used_pmids)
    if len(new_pmids) > 10000:
        new_pmids = random.choices(new_pmids, k=10000)
    _, contains = additional_entities[entity]
    text_dict = get_plaintexts_for_pmids(new_pmids, contains=contains)
    corpus.extend([(text, entity, pmid) for pmid, text in text_dict.items() if len(text) >= 5])

In [23]:
names.update({key: value[0] for key, value in additional_entities.items()})
names.update({key: value[0] for key, value in unambiguous_agent_texts.items()})
pos_labels = list(set(pos_labels) | additional_entities.keys() |
                  unambiguous_agent_texts.keys())

In [17]:
%%capture

classifier = AdeftClassifier(shortforms, pos_labels=pos_labels, random_state=1729)
param_grid = {'C': [100.0], 'max_features': [10000]}
texts, labels, pmids = zip(*corpus)
classifier.cv(texts, labels, param_grid, cv=5, n_jobs=5)

INFO: [2020-11-10 02:52:22] /adeft/PP/adeft/adeft/modeling/classify.py - Beginning grid search in parameter space:
{'C': [100.0], 'max_features': [10000]}
INFO: [2020-11-10 02:53:44] /adeft/PP/adeft/adeft/modeling/classify.py - Best f1 score of 0.9915721160418993 found for parameter values:
{'logit__C': 100.0, 'tfidf__max_features': 10000}


In [18]:
classifier.stats

{'label_distribution': {'FPLX:LH': 4183,
  'MESH:D007026': 280,
  'ungrounded': 128,
  'PLOD': 15,
  'MESH:D008138': 3,
  'CHEBI:CHEBI:61051': 6,
  'CHEBI:CHEBI:50512': 3,
  'PUBCHEM:161464': 8,
  'MESH:D018546': 1,
  'MESH:C533735': 2,
  'MESH:D006495': 3,
  'MESH:C000588632': 5},
 'f1': {'mean': 0.991572, 'std': 0.001366},
 'precision': {'mean': 0.984913, 'std': 0.002687},
 'recall': {'mean': 0.998327, 'std': 0.001219},
 'MESH:D018546': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'PLOD': {'f1': {'mean': 1.0, 'std': 0.0},
  'pr': {'mean': 1.0, 'std': 0.0},
  'rc': {'mean': 1.0, 'std': 0.0}},
 'MESH:C533735': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean': 0.0, 'std': 0.0},
  'rc': {'mean': 0.0, 'std': 0.0}},
 'FPLX:LH': {'f1': {'mean': 0.991572, 'std': 0.001366},
  'pr': {'mean': 0.998327, 'std': 0.001219},
  'rc': {'mean': 0.984913, 'std': 0.002687}},
 'PUBCHEM:161464': {'f1': {'mean': 0.0, 'std': 0.0},
  'pr': {'mean':

In [19]:
disamb = AdeftDisambiguator(classifier, grounding_dict, names)

In [20]:
disamb.dump(model_name, results_path)

In [21]:
print(disamb.info())

Disambiguation model for LH

Produces the disambiguations:
	Heparin, Low-Molecular-Weight	MESH:D006495
	Hypocotyl	MESH:D018546
	Hypothalamic Area, Lateral	MESH:D007026
	LH*	FPLX:LH
	Leonurine	PUBCHEM:161464
	Loop of Henle	MESH:D008138
	PLOD	PLOD
	lianhuaqingwen	MESH:C000588632
	lidocaine hydrochloride	CHEBI:CHEBI:50512
	lipid hydroperoxide	CHEBI:CHEBI:61051
	liposomal honokiol	MESH:C533735

Class level metrics:
--------------------
Grounding                    	Count	F1     
                           LH*	4183	0.99157
   Hypothalamic Area, Lateral	 280	 0.9398
                   Ungrounded	 128	0.84824
                         PLOD	  15	    1.0
                    Leonurine	   8	    0.0
          lipid hydroperoxide	   6	0.33333
               lianhuaqingwen	   5	    0.6
                Loop of Henle	   3	    0.0
      lidocaine hydrochloride	   3	    0.0
Heparin, Low-Molecular-Weight	   3	    0.0
           liposomal honokiol	   2	    0.0
                    Hypocotyl	   1	    0.0

We

In [22]:
model_to_s3(disamb)

In [30]:
preds = [disamb.disambiguate(text) for text in all_texts.values()]

In [31]:
texts = [text for pred, text in zip(preds, all_texts.values()) if pred[0] == 'HGNC:10967']

In [36]:
texts[3]

'The non-neuronal monoamine transporters (OCT1, OCT2, EMT, and PMAT) play a key role in the clearance of monoamines from extracellular compartments. In a previous report we described endometrial distribution and cyclic variation of the vesicular monoamine transporter (VMAT2) mRNA and the neuronal norepinephrine transporter (NET) mRNA. In the present study we used in situ hybridization, real-time PCR and immunohistochemistry to reveal tissue distribution and cyclic variation of mRNA for the non-neuronal monoamine transporters in the human endometrium and early pregnancy decidua. We found that non-neuronal monoamine transporters are predominantly expressed in the stroma. The plasma membrane monoamine transporter (PMAT) mRNA expression peaked in the proliferative phase, whereas the extra-neuronal monoamine transporter (EMT) mRNA expression peaked in the secretory phase. The organic cation transporter 2 (OCT2) mRNA expression was exclusively detected in few scattered stromal cells and OCT1